In [13]:
import jax.numpy as jnp
from jax import jit, grad, random

from jax.example_libraries import optimizers
from jax.example_libraries import stax
from jax.example_libraries.stax import Dense, Relu, LogSoftmax, Tanh

import time
import itertools

import gym

import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt

In [3]:
env = gym.make("CartPole-v1", render_mode="human")
obs = env.reset()
done = False

while not done:
    act = env.action_space.sample()
    obs, rew, done, _, _ = env.step(act)
    env.render()

env.close()

/home/alex/miniconda3/envs/ml/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [4]:
predict(init_params, obs.reshape(1,4))

NameError: name 'predict' is not defined

In [109]:
random.categorical(rng, predict(init_params, obs.reshape(1, 4)), shape=(5,5))

Array([[0, 1, 0, 1, 1],
       [0, 1, 0, 1, 1],
       [1, 0, 1, 1, 0],
       [0, 0, 0, 1, 0],
       [1, 1, 1, 1, 1]], dtype=int32)

In [114]:
random.categorical(rng, predict(init_params, obs.reshape(1, 4)), shape=(1, 1)).item()

1

In [15]:
batch_size = 4096
step_size = 0.01

In [16]:
rng = random.PRNGKey(0)

In [17]:
init_random_params, predict = stax.serial(
    Dense(32), Tanh,
    Dense(2), LogSoftmax)

In [18]:
opt_init, opt_update, get_params = optimizers.adam(step_size)

In [19]:
def loss(params, batch):
  obs, act, weights = batch # obs: (m, 4), act: (m, 2), weights: (m, 1)
  preds = predict(params, obs) # preds = log action probs, (m, 2)

  logp = jnp.sum(jnp.multiply(preds, act), axis=1, keepdims=True) # log probs des actions choisies pendant la traj. (m, 1)

  return -jnp.mean(jnp.multiply(logp, weights))

In [20]:
@jit
def one_hot(y):
    return jnp.array(y == (jnp.arange(2).reshape(2, 1)), dtype=jnp.float32)

In [21]:
@jit
def update(i, opt_state, batch):
    params = get_params(opt_state)
    return opt_update(i, grad(loss)(params, batch), opt_state)

In [22]:
_, init_params = init_random_params(rng, (-1, 4))
opt_state = opt_init(init_params)
itercount = itertools.count()

In [23]:
def one_epoch():
    global opt_state

    batch_obs = []
    batch_act = []
    batch_weights = []
    batch_rets = []
    batch_lens = []

    rng = random.PRNGKey(0)

    env = gym.make("CartPole-v1")
    obs, _ = env.reset()
    done = False
    ep_rews = []

    while True:
        batch_obs.append(obs.copy())

        rng, key_act = random.split(rng)
        act = random.categorical(key_act, predict(init_params, obs.reshape(1, 4)), shape=(1, 1)).item()

        obs, rew, done, _, _ = env.step(act)

        batch_act.append(act) #attention a la shape finale de batch_act
        ep_rews.append(rew)

        if done:
            ep_ret = sum(ep_rews)
            ep_len = len(ep_rews)
            
            batch_rets.append(ep_ret)
            batch_lens.append(ep_len)

            batch_weights += [ep_ret] * ep_len

            obs, _ = env.reset()
            done = False
            ep_rews = []

            if len(batch_obs) > batch_size:
                break
    
    #print(jnp.array(batch_obs).shape)
    #print(jnp.array(batch_act).shape)
    #return jnp.array(batch_act)

    opt_state = update(next(itercount), opt_state, (jnp.array(batch_obs), one_hot(jnp.array(batch_act)).T, jnp.array(batch_weights).reshape(-1, 1)))
    
    return jnp.array(batch_rets), jnp.array(batch_lens)
        

In [24]:
#batch_rets, batch_lens = one_epoch()

In [25]:
for i in range(50):
    batch_rets, batch_lens = one_epoch()
    print('epoch: %3d \t return: %.3f \t ep_len: %.3f'% (i, jnp.mean(batch_rets), jnp.mean(batch_lens)))

/home/alex/miniconda3/envs/ml/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


epoch:   0 	 return: 25.981 	 ep_len: 25.981
epoch:   1 	 return: 28.366 	 ep_len: 28.366
epoch:   2 	 return: 26.740 	 ep_len: 26.740
epoch:   3 	 return: 25.681 	 ep_len: 25.681
epoch:   4 	 return: 27.564 	 ep_len: 27.564
epoch:   5 	 return: 25.619 	 ep_len: 25.619
epoch:   6 	 return: 26.628 	 ep_len: 26.628
epoch:   7 	 return: 28.144 	 ep_len: 28.144
epoch:   8 	 return: 25.956 	 ep_len: 25.956
epoch:   9 	 return: 25.146 	 ep_len: 25.146
epoch:  10 	 return: 27.682 	 ep_len: 27.682
epoch:  11 	 return: 27.039 	 ep_len: 27.039


KeyboardInterrupt: 